<a href="https://colab.research.google.com/github/fernando2393/DT2119-Final-Project/blob/master/MusicGenreClassification_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
from google.colab import files
files.upload()

Saving constants.py to constants (1).py
Saving features.py to features (1).py
Saving loader.py to loader (1).py
Saving main.py to main (1).py


{'constants.py': b'DATASETS = "/content/drive/My Drive/Datasets/fma_small/"\r\nMETADATA = "/content/drive/My Drive/Datasets/fma_metadata/"\r\nMFCC = "/content/drive/My Drive/Datasets/mfcc.csv"\r\nSPECTROGRAM = "/content/drive/My Drive/Datasets/spectrogram.csv"\r\n',
 'features.py': b'import pandas as pd\r\nimport sklearn\r\nimport librosa.display\r\nimport librosa\r\nimport os\r\nfrom tqdm import tqdm\r\nimport warnings\r\nimport constants as cts\r\n\r\n\r\ndef compute_feature(mode, filepath):\r\n    """\r\n    This method loads the a music track and computes the mfcc.\r\n    :param mode: allows to get either spectrogram or mfcc.\r\n    :param filepath: music track.\r\n    :return mfcc of the data track.\r\n    """\r\n    y, sr = librosa.load(filepath, duration=28.5, sr=44100, mono=True, offset=0.5)  # Load 28 seconds\r\n    # (same length for every track)\r\n    if mode == \'spectrogram\':\r\n        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)\r\n        spect

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, ZeroPadding2D, ELU, Reshape, GRU
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
import loader
(x_train_prelim, y_train_prelim), (x_val_prelim, y_val_prelim), (x_test_prelim, y_test_prelim) = loader.get_train_val_test(mode="spectrogram")
print("There are the following classes:")
classes = set(y_train_prelim.tolist()) & set(y_val_prelim.tolist()) & set(y_test_prelim.tolist())
print(classes)

Calculating spectrogram...


OSError: ignored

In [0]:
# remove corrupted samples
x_train_prelim = np.delete(x_train_prelim, 3495)
x_train_prelim = np.delete(x_train_prelim, 3495)
x_train_prelim = np.delete(x_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)
y_train_prelim = np.delete(y_train_prelim, 3495)

In [0]:
x_train = []
for i in range(6397 - 3):
  x_train[i].append(x_train_prelim[i])
del x_train_prelim

x_val = []
x_test = []

for i in range(800):
  x_val[i].append(x_val_prelim[i])
  x_test[i].append(x_test_prelim[i])

del x_val_prelim
del x_test_prelim

In [0]:
x_train = x_train.reshape((6394, 13, 2455, 1))
x_val = x_val.reshape((800, 13, 2455, 1))
x_test = x_test.reshape((800, 13, 2455, 1))

In [0]:
dict_labels = {'Electronic': 0, 'Experimental': 1, 'Folk': 2, 'Hip-Hop': 3,
               'Instrumental': 4, 'International': 5, 'Pop': 6, 'Rock': 7}

y_train = np.zeros([6397 - 3])
y_val = np.zeros([800])
y_test = np.zeros([800])

for i in range(len(y_train)):
  y_train[i] = dict_labels[y_train_prelim[i]]

In [0]:
for i in range(len(y_val_prelim)):
  y_val[i] = dict_labels[y_val_prelim[i]]
  y_test[i] = dict_labels[y_test_prelim[i]]

del y_train_prelim
del y_val_prelim
del y_test_prelim

In [0]:
# One-hot encoding of classes
y_train = to_categorical(y_train, num_classes=8)
y_val = to_categorical(y_val, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

In [0]:
x_train.shape

In [0]:
y_train[0]

In [0]:
class Classifier:
    @staticmethod
    def build(width, height, depth, classes):
        
        model = Sequential()
        inputShape = (height, width, depth)

        model.add(Conv2D(16, (3, 3), strides=(1, 1), input_shape=inputShape)) 
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        
        model.add(Conv2D(32, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        
        model.add(Conv2D(64, (3, 3), strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(Dropout(0.2))
        
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(classes, activation='softmax'))
        
        print("summary")
        print(model.summary())
        
        return model

In [0]:
EPOCHS = 25
INIT_LR = 0.005
BS = 16

model = Classifier.build(width=2455, height=128, depth=1, classes=8)

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# Callbacks: early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1, 
                               patience=10,
                               mode='max',
                               restore_best_weights=True)

filepath = "weights.{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [early_stopping, checkpoint]

In [0]:
history = model.fit(x_train, y_train, batch_size=BS,
                    validation_data=(x_val, y_val),
                   steps_per_epoch=len(x_train) // BS,
                    callbacks=callbacks_list,
                    epochs=EPOCHS, verbose=1)

In [0]:
plt.pcolormesh(x_test_prelim[300])